### INSTALACION Y CONFIGURACION

In [1]:
%pip install langchain-community langchain-core langchain-text-splitters langchain-ollama langchain-chroma langchain-google-genai pypdf python-dotenv ipywidgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\IPF-2025\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


1. Implementacion practica de RAG

Importaciones

In [2]:
import os
from getpass import getpass

# Importaciones del código de ejemplo
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

# Importaciones para el resto del trabajo
from langchain_community.llms import Ollama 

In [3]:
load_dotenv() 
# Usemos el estándar de Google para que funcione.
if "GOOGLE_API_KEY" not in os.environ:
    try:
        # Intenta cargar "API_KEY" como en tu ejemplo
        os.environ["GOOGLE_API_KEY"] = os.getenv("API_KEY")
        if os.environ["GOOGLE_API_KEY"] is None:
            raise Exception
    except:
        os.environ["GOOGLE_API_KEY"] = getpass("Ingresa tu Google API Key: ")
        
print("API Key de Google configurada.")

API Key de Google configurada.


Definicion de funciones

In [5]:
def upload_pdf(url: str):    
    try:
        loader = PyPDFLoader(url)
        loader_iter = loader.lazy_load() 
        text = ""
        for page in loader_iter: 
            text += page.page_content + "\n"
        return text
    except Exception as e:
        print(f"Error cargando el PDF: {e}")
        return ""

# --- Función de Splitter ---
def text_splitter(text): 
    text_splitter = CharacterTextSplitter(
        chunk_size = 2000,
        chunk_overlap = 100,
        separator="\n"
    )
    texts = text_splitter.create_documents([text])
    return texts

# --- Embedding Global (del ejemplo) ---
print("Cargando modelo de embedding 'nomic-embed-text'...")
embedding = OllamaEmbeddings(
    model = "nomic-embed-text"
)
print("Embedding listo.")

# --- Función de Cargar Vector Store (del ejemplo) ---
def get_vector_store(name_collection: str): 
    vector_store = Chroma(
        collection_name= name_collection,
        embedding_function=embedding, # Usa el embedding global
        persist_directory="./vectorstore"
    )
    return vector_store

# --- Función de Retrieval (del ejemplo) ---
def retrieval(input_user: str): 
    vector_store = get_vector_store("langchain") # Carga la DB
    docs = vector_store.similarity_search(input_user)
    return docs

# --- Prompt Global (del ejemplo) ---
prompt = PromptTemplate.from_template("""
    Eres un asistente encargado de responder pregutnas sobre Langchain.
    Utiliza siempre el contexto para responder.
    contexto = {contexto}
    pregunta del usuario: {input_user}
""")

# --- Función de Response (del ejemplo) ---
def response(input_user: str, contexto: str):
    api_key_google = os.getenv("GOOGLE_API_KEY") # Usamos la key correcta
    llm = ChatGoogleGenerativeAI(
        api_key=api_key_google,
        model="gemini-2.0-flash", # Modelo del ejemplo
        temperature= 0.5
    )
    
    # Formateamos el prompt con los datos
    prompt_formateado = prompt.format(contexto=contexto, input_user=input_user)

    # Devolvemos el stream
    for chunk in llm.stream(prompt_formateado):
        yield chunk.content

print("\n¡Todas las funciones base han sido definidas!")

Cargando modelo de embedding 'nomic-embed-text'...
Embedding listo.

¡Todas las funciones base han sido definidas!


In [6]:
# --- FIX 1: Función para CREAR la base de datos ---
def create_vector_store(texts, name_collection: str):
    """
    Toma los documentos 'spliteados' y los indexa en Chroma.
    """
    print(f"Creando/Actualizando la colección: {name_collection}")
    # Chroma.from_documents CREA la base de datos y la guarda
    vector_store = Chroma.from_documents(
        documents=texts,
        embedding=embedding, # Usa el embedding global
        collection_name=name_collection,
        persist_directory="./vectorstore" # La guarda en disco
    )
    print("¡Indexación completa!")
    return vector_store

# --- FIX 2: Función para convertir la LISTA [docs] en un STRING ---
def format_docs_to_context(docs: list):
    """
    Convierte la lista de documentos de 'retrieval' en un solo string
    para pasarlo al 'contexto' del prompt.
    """
    return "\n\n---\n\n".join([doc.page_content for doc in docs])

print("Funciones 'Fixer' listas.")

Funciones 'Fixer' listas.


In [7]:
PDF = "Nace Wikileaks (2006) - Ciberseguridad (1).pdf" 

print("--- Fase de Indexación ---")

# 1. Cargar PDF
# (Usamos la función 'upload_pdf' que definimos antes)
raw_text = upload_pdf(PDF)

if raw_text:
    print(f"PDF '{PDF}' cargado en memoria.")
    
    # 2. Splitear
    # (Usamos la función 'text_splitter' que definimos)
    texts_chunks = text_splitter(raw_text)
    print(f"Documento dividido en {len(texts_chunks)} chunks.")
    
    # 3. Indexar
    # (Usamos nuestra función 'create_vector_store' para guardar los chunks)
    create_vector_store(texts_chunks, "langchain")
    print("Base de datos creada en ./vectorstore")
else:
    print(f"Error: No se pudo cargar el PDF '{PDF}'.")
    print("Por favor, verifica que el nombre del archivo es correcto y que está en la misma carpeta que el notebook.")

--- Fase de Indexación ---
PDF 'Nace Wikileaks (2006) - Ciberseguridad (1).pdf' cargado en memoria.
Documento dividido en 39 chunks.
Creando/Actualizando la colección: langchain
¡Indexación completa!
Base de datos creada en ./vectorstore


In [8]:
# Celda de Código
# --- Flujo de Pregunta/Respuesta ---

input_user = input("Hace una pregunta sobre Wikileaks: ")

# 1. Retrieval
print("Recuperando documentos...")
docs = retrieval(input_user)
print(f"Documentos recuperados: {len(docs)}")

# 2. Formatear Contexto (Usando nuestra función FIX 2)
contexto_str = format_docs_to_context(docs)

# 3. Response (Función del profesor)
print("\nRespuesta:")
for chunk in response(input_user= input_user, contexto= contexto_str):
    print(chunk, end="", flush=True)
print("\n")

Recuperando documentos...
Documentos recuperados: 4

Respuesta:
Julian Assange es el creador de WikiLeaks, una página web que publica documentos e imágenes filtrados con el fin de revelar escándalos y casos de corrupción que consideran de interés público. Nació el 3 de julio de 1971 en Queensland, Australia. De adolescente se convirtió en un hábil hacker. Es partidario de la libre circulación de información.



2. Exploración de Modelos de Embedding y LLM:

In [9]:
# --- Definición de Funciones para Embedding Alternativo ---

def get_embedding_alternative(model_name="mxbai-embed-large"):
    """
    Carga un modelo de embedding alternativo de Ollama.
    """
    print(f"Cargando modelo de embedding '{model_name}'...")
    try:
        alt_embedding = OllamaEmbeddings(model=model_name)
        print("Embedding alternativo listo.")
        return alt_embedding
    except Exception as e:
        print(f"Error: {e}. Asegúrate de tener Ollama corriendo y el modelo '{model_name}' descargado.")
        return None

def create_vector_store_alternative(texts, name_collection: str, alt_embedding):
    """
    Crea una NUEVA colección con el embedding alternativo.
    """
    print(f"Creando/Actualizando la colección alternativa: {name_collection}")
    # Usamos el embedding nuevo y un directorio nuevo
    vector_store = Chroma.from_documents(
        documents=texts,
        embedding=alt_embedding, # <-- Usa el embedding nuevo
        collection_name=name_collection,
        persist_directory=f"./{name_collection}_db" # Nuevo directorio
    )
    print(f"¡Indexación alternativa completa en ./{name_collection}_db!")
    return vector_store

def retrieval_alternative(input_user: str, name_collection: str, alt_embedding): 
    """
    Busca en la base de datos alternativa.
    """
    vector_store = Chroma(
        collection_name=name_collection,
        embedding_function=alt_embedding,
        persist_directory=f"./{name_collection}_db"
    )
    docs = vector_store.similarity_search(input_user)
    return docs

print("Funciones para el embedding alternativo listas.")

Funciones para el embedding alternativo listas.


In [10]:
# --- Prueba del Embedding Alternativo ---

print("--- Probando Embedding 'mxbai-embed-large' ---")

# 1. Cargar el modelo de embedding
alt_emb_model = get_embedding_alternative()

if alt_emb_model:
    # 2. Indexar (usando los chunks del Punto 1, "texts_chunks")
    create_vector_store_alternative(texts_chunks, "langchain_alt", alt_emb_model)

    # 3. Probar el RAG
    input_user = input("Human (prueba embedding alt): ")
    
    # 4. Retrieval (de la nueva DB)
    docs_alt = retrieval_alternative(input_user, "langchain_alt", alt_emb_model)
    print(f"Documentos recuperados (mxbai): {len(docs_alt)}")
    
    # 5. Formatear
    contexto_alt = format_docs_to_context(docs_alt)
    
    # 6. Response (con el LLM original)
    print("\nIA (Gemini-Flash con embedding 'mxbai'):")
    for chunk in response(input_user=input_user, contexto=contexto_alt):
        print(chunk, end="", flush=True)
    print("\n")
else:
    print("No se pudo cargar el embedding alternativo. Saltando prueba.")

--- Probando Embedding 'mxbai-embed-large' ---
Cargando modelo de embedding 'mxbai-embed-large'...
Embedding alternativo listo.
Creando/Actualizando la colección alternativa: langchain_alt
¡Indexación alternativa completa en ./langchain_alt_db!
Documentos recuperados (mxbai): 4

IA (Gemini-Flash con embedding 'mxbai'):
Julian Assange nació el 3 de julio de 1971 en Queensland, Australia. De adolescente y a los 20 años, se convirtió en un hábil hacker, conocido en la comunidad australiana de piratas informáticos como Mendax. En 1996, se declaró culpable de 24 delitos de piratería informática. Es partidario de la libre circulación de información y registró el nombre de dominio Wikileaks.org en 1999, aunque no lo empezó a usar activamente hasta 2006.




In [13]:
# --- 2.2. Experimento: Comparando LLMs (Flash vs TinyLlama) ---

# --- 1. Definimos una función de response para TinyLlama (Ollama) ---
def response_tinyllama(input_user: str, contexto: str):
    """
    Usa un modelo local de Ollama muy ligero: 'tinyllama'.
    """
    print("\n(Usando LLM local TinyLlama...)")
    try:
        llm = Ollama(model="tinyllama")
        prompt_formateado = prompt.format(contexto=contexto, input_user=input_user)
        # .stream() para Ollama devuelve strings
        for chunk_str in llm.stream(prompt_formateado):
            yield chunk_str
    except Exception as e:
        print(f"Error: {e}. ¿Ollama está corriendo y 'tinyllama' descargado?")
        yield ""

print("Función para LLM alternativo ('tinyllama') lista.")

# --- 2. Prueba de Comparación de LLMs ---
print("\n--- Probando LLMs Alternativos (Flash vs TinyLlama) ---")
input_user = input("Human (prueba LLMs): ")

# Usamos el retrieval ORIGINAL (Punto 1, 'nomic-embed-text')
print("Recuperando documentos (con 'nomic-embed-text')...")
docs = retrieval(input_user) 
contexto_str = format_docs_to_context(docs)

# --- PRUEBA 1: MODELO ORIGINAL ---
print("\n--- IA (Respuesta con 'gemini-2.0-flash' - Original): ---")
# Usamos la función 'response' que definimos en el Punto 1.1
for chunk in response(input_user=input_user, contexto=contexto_str):
    print(chunk, end="", flush=True)
print("\n")

# --- PRUEBA 2: MODELO ALTERNATIVO ---
print("\n--- IA (Respuesta con 'tinyllama' - Alternativo): ---")
# Usamos la nueva función 'response_tinyllama'
for chunk in response_tinyllama(input_user=input_user, contexto=contexto_str):
    print(chunk, end="", flush=True)
print("\n")

Función para LLM alternativo ('tinyllama') lista.

--- Probando LLMs Alternativos (Flash vs TinyLlama) ---
Recuperando documentos (con 'nomic-embed-text')...

--- IA (Respuesta con 'gemini-2.0-flash' - Original): ---
Julian Assange es el creador de WikiLeaks, un sitio web que publica documentos e imágenes filtrados para revelar escándalos y casos de corrupción de interés público. Nació el 3 de julio de 1971 en Queensland, Australia, y en su adolescencia se convirtió en un hábil hacker. Es partidario de la libre circulación de información y registró el dominio Wikileaks.org en 1999, aunque no lo usó activamente hasta 2006.



--- IA (Respuesta con 'tinyllama' - Alternativo): ---

(Usando LLM local TinyLlama...)


C:\Users\IPF-2025\AppData\Local\Temp\ipykernel_26308\3866346161.py:10: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(model="tinyllama") # <-- ¡AQUÍ ESTÁ EL CAMBIO!


Julian Assange is the founder and editor-in-chief of WikiLeaks, which launched in 2006 as a website for publishing documents and information obtained by whistleblowers. The site was initially known as "Wiki," but its name was later changed to reflect its mission of disseminating leaked information.

WikiLeaks is not just a website; it is an activism group that promotes transparency, accountability, and the right to access information held by governments and corporations. The site has played a critical role in exposing corruption, human rights abuses, and environmental crimes around the world.

In 2009, WikiLeaks published the "Collateral Murder" video, which showed U.S. Soldiers executing an unarmed Afghan civilian near a checkpoint during a raid in 2007. The video quickly gained international attention and sparked protests around the world.

WikiLeaks has been criticized by authorities for publishing secrets obtained through illegal means, but the organization has consistently maintai

## Análisis de Modelos

### Comparación de Modelos de Embedding

* **`nomic-embed-text` (Base):**
    * La respuesta fue una **visión general y conceptual**.
    * Se centró en definir *qué es WikiLeaks* ("creador de WikiLeaks, una página web que publica...") y luego dio datos biográficos básicos (fecha de nacimiento, hacker, partidario de la libre información).
    * Esto sugiere que `nomic` entendió la pregunta "Quién es Julian Assange" y la relacionó con el concepto general de "WikiLeaks".

* **`mxbai-embed-large` (Alternativo):**
    * La respuesta fue **biográfica, específica y mucho más detallada**.
    * **Omitió la definición** de WikiLeaks y se centró 100% en la persona.
    * Recuperó hechos concretos que el otro modelo pasó por alto, como su alias de hacker ("Mendax"), su condena penal en 1996 ("24 delitos de piratería") y la línea de tiempo exacta del dominio (registrado en 1999, usado en 2006).

* **Conclusión:**
    `mxbai-embed-large`, al ser un modelo más "grande" (large), demostró una capacidad superior para realizar una **búsqueda semántica más granular y precisa**. No solo encontró "Assange", sino que encontró los chunks con los detalles biográficos más específicos.

    Sin embargo, esta mayor precisión tuvo un **costo de rendimiento extremo**: la indexación con `mxbai` fue significativamente más lenta (tardó más de 15 minutos) en comparación con `nomic-embed-text` (que tardó 6 minutos). Esto demuestra el compromiso clásico entre la velocidad de indexación y la calidad de la recuperación.

### Comparación de Modelos de LLM (Punto 2.2)

* **`gemini-2.0-flash` (Original - Google) vs. `tinyllama` (Alternativo - Ollama):**
* **Adherencia al Contexto:** `gemini-2.0-flash` fue excelente. Dio una respuesta corta y precisa (mencionando fechas como 1971, 1999, 2006) que probó que estaba leyendo el PDF de Wikileaks.
* **Calidad de Respuesta:** `tinyllama` **falló en seguir la instrucción** "Utiliza siempre el contexto". En lugar de usar los chunks recuperados, dio una respuesta genérica sobre Wikileaks (mencionando "Collateral Murder", etc.) que probablemente venía de su conocimiento general pre-entrenado.
* **Idioma:** `gemini-2.0-flash` detectó el idioma del prompt (español) y respondió correctamente. `tinyllama`, al ser un modelo base entrenado principalmente en inglés, ignoró el idioma del prompt y respondió en inglés.
* **Conclusión:** Para RAG, un modelo que sigue bien las instrucciones (como `gemini-2.0-flash`) es mucho mejor que un modelo que "sabe más" pero no obedece el prompt (como `tinyllama`).

3. Optimización del Separador de Texto:

In [14]:
# --- Definición de Nuevos Splitters y Retrievers ---

# --- 1. Splitter con chunks más pequeños ---
def text_splitter_small(text): 
    """
    Usa el splitter original pero con un chunk_size mucho más pequeño.
    """
    text_splitter = CharacterTextSplitter(
        chunk_size = 500,   # <-- ¡MÁS PEQUEÑO! (Antes 2000)
        chunk_overlap = 50, # (Un overlap más pequeño)
        separator="\n"
    )
    texts = text_splitter.create_documents([text])
    return texts

# --- 2. Splitter Recursivo (El más recomendado) ---
def text_splitter_recursive(text):
    """
    Usa RecursiveCharacterTextSplitter, que es más inteligente
    al separar por párrafos ("\n\n"), luego frases ("\n"), etc.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1000, # Un tamaño de chunk razonable
        chunk_overlap = 150
    )
    texts = text_splitter.create_documents([text])
    return texts

# --- 3. Nuevas funciones de Retrieval ---
# (Necesitamos esto para apuntar a las nuevas colecciones)

def retrieval_small(input_user: str): 
    """
    Busca en la base de datos creada con chunks pequeños.
    """
    # get_vector_store usa el embedding original (nomic), ¡lo cual es correcto!
    vector_store = get_vector_store("langchain_small") # <-- Nueva colección
    docs = vector_store.similarity_search(input_user)
    return docs

def retrieval_recursive(input_user: str): 
    """
    Busca en la base de datos creada con el splitter recursivo.
    """
    vector_store = get_vector_store("langchain_recursive") # <-- Nueva colección
    docs = vector_store.similarity_search(input_user)
    return docs

print("Funciones para splitters alternativos listas.")

Funciones para splitters alternativos listas.


In [15]:
# Celda de Código
# ---  Prueba del Splitter Pequeño (chunk_size=500) ---

print("--- Probando Splitter Pequeño (chunk_size=500) ---")

# 1. Cargar el PDF 
raw_text = upload_pdf("Nace Wikileaks (2006) - Ciberseguridad (1).pdf") 

# 2. Splitear con la nueva función
texts_chunks_small = text_splitter_small(raw_text)
print(f"Documento dividido en {len(texts_chunks_small)} chunks pequeños.")
print(f"(Comparado con los {len(texts_chunks)} chunks originales de 2000c).")

# 3. Indexar en una NUEVA colección
# (Esto usará el embedding 'nomic' original, lo cual es correcto)
print("Indexando chunks pequeños...")
create_vector_store(texts_chunks_small, "langchain_small")

# 4. Probar el RAG con el nuevo splitter
input_user = input("Human (prueba chunks pequeños): ")

# 5. Retrieval (de la nueva DB 'langchain_small')
print("Recuperando chunks pequeños...")
docs_small = retrieval_small(input_user)
print(f"Documentos recuperados: {len(docs_small)}")

# 6. Formatear y Responder
contexto_small = format_docs_to_context(docs_small)

print("\nIA (Respuesta con chunks de 500c):")
for chunk in response(input_user=input_user, contexto=contexto_small):
    print(chunk, end="", flush=True)
print("\n")

--- Probando Splitter Pequeño (chunk_size=500) ---


Created a chunk of size 960, which is longer than the specified 500
Created a chunk of size 735, which is longer than the specified 500
Created a chunk of size 767, which is longer than the specified 500
Created a chunk of size 501, which is longer than the specified 500


Documento dividido en 162 chunks pequeños.
(Comparado con los 39 chunks originales de 2000c).
Indexando chunks pequeños...
Creando/Actualizando la colección: langchain_small
¡Indexación completa!
Recuperando chunks pequeños...
Documentos recuperados: 4

IA (Respuesta con chunks de 500c):
Julian Assange es una figura clave en el periodismo de investigación, conocido por fundar WikiLeaks y colaborar con medios internacionales en la publicación de documentos filtrados, como los Diarios de Guerra de Afganistán y los Papeles de Irak. Enfrenta un largo proceso legal bajo la Ley de Espionaje de Estados Unidos y debates internacionales sobre su extradición. En 1996, fue multado y puesto en libertad tras ser juzgado por delitos informáticos en el Condado de Victoria, Melbourne.




In [16]:
# --- 3.2. Prueba del Splitter Recursivo ---

print("--- Probando Splitter Recursivo (chunk_size=1000) ---")

# 2. Splitear con la nueva función "inteligente"
texts_chunks_recursive = text_splitter_recursive(raw_text)
print(f"Documento dividido en {len(texts_chunks_recursive)} chunks recursivos.")
print(f"(Comparado con los {len(texts_chunks_small)} chunks pequeños).")

# 3. Indexar en una NUEVA colección
print("Indexando chunks recursivos...")
create_vector_store(texts_chunks_recursive, "langchain_recursive")

# 4. Probar el RAG con el nuevo splitter
input_user = input("Human (prueba chunks recursivos): ")

# 5. Retrieval (de la nueva DB 'langchain_recursive')
print("Recuperando chunks recursivos...")
docs_recursive = retrieval_recursive(input_user)
print(f"Documentos recuperados: {len(docs_recursive)}")

# 6. Formatear y Responder
contexto_recursive = format_docs_to_context(docs_recursive)

print("\nIA (Respuesta con chunks recursivos):")
for chunk in response(input_user=input_user, contexto=contexto_recursive):
    print(chunk, end="", flush=True)
print("\n")

--- Probando Splitter Recursivo (chunk_size=1000) ---
Documento dividido en 85 chunks recursivos.
(Comparado con los 162 chunks pequeños).
Indexando chunks recursivos...
Creando/Actualizando la colección: langchain_recursive
¡Indexación completa!
Recuperando chunks recursivos...
Documentos recuperados: 4

IA (Respuesta con chunks recursivos):
Julian Assange es un hábil hacker australiano, conocido en la comunidad de piratas informáticos como Mendax. Es partidario de la libre circulación de información y fundó WikiLeaks, un sitio web que transformó en un lugar seguro para denunciantes. Sin embargo, sus acciones han generado controversia y críticas, siendo acusado de poner en peligro vidas y dañar la seguridad nacional por figuras políticas.




## Análisis de Separadores
### 1. `CharacterTextSplitter (chunk_size=2000)` - Original
* **Chunks Creados:** 39
* **Análisis:** Los chunks eran demasiado grandes (2000 caracteres). Esto introdujo mucho "ruido" en el contexto recuperado. La respuesta del LLM fue muy general y biográfica ("Nació en 1971..."), fallando en capturar los detalles más relevantes del trabajo de Assange.

### 2. `CharacterTextSplitter (chunk_size=500)` - Pequeño
* **Chunks Creados:** 162
* **Análisis:** La calidad de la respuesta **mejoró drásticamente**. Al tener chunks más pequeños, el sistema recuperó fragmentos láser-enfocados, permitiendo al LLM dar una respuesta muy específica y de alta calidad (mencionando "Diarios de Guerra", "Ley de Espionaje", "multado en 1996").
* **Problema:** Este splitter mostró *warnings* (`Created a chunk of size 960...`). Demostró que no es "inteligente", ya que al intentar respetar el separador `\n`, tuvo que romper la regla de `chunk_size=500`. No es un splitter confiable.

### 3. `RecursiveCharacterTextSplitter (chunk_size=1000)` - Recursivo
* **Chunks Creados:** 85
* **Análisis:** Este splitter dio la **mejor respuesta general**. Fue **balanceada y coherente**, capturando la identidad ("Mendax"), la misión ("denunciantes") y el conflicto ("controversia", "poner en peligro vidas").
* **Por qué es mejor:** Es un splitter "inteligente". Intenta cortar semánticamente (por párrafos, luego frases), lo que resulta en chunks que tienen más sentido. Respetó el `chunk_size` sin generar *warnings*. Esto le dio al LLM un contexto de mucha mayor calidad.